# PAMAP2 Physical Activity Monitoring データの分析サンプル（後半・途中までは前回説明済み）

### 準備作業（データ読み込み等）

必要なライブラリのインポート

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import math
from sklearn.model_selection import train_test_split


すべてのファイルからデータを読み込み，データフレームを作成する．ここでは，ファイルをリスト化し，カラムの各列のヘッダを作成する．
##### ・ファイル名のリスト作成：各被験者のデータファイルを読み込むためにファイル名をリスト化
##### ・活動ラベルの辞書作成：各活動の番号（ラベル）と名前（例：1=lying，2=sitting）を辞書にマッピング
##### ・IMUカテゴリのリスト作成：各センサー（手首，胸，足首）のデータ列名を別々にリスト化
##### ・カラムコレクションの統合：タイムスタンプ，活動ラベル，心拍数，各IMUカテゴリのデータ列名をすべて結合し，データフレームのカラム名を作成
最後に，作成したヘッダに対して列数を表示する．各データファイルは54列あるので，それと等しくなっていればよい．

In [ ]:
# Load data
list_of_files = ['PAMAP2_Dataset/Protocol/subject101.dat',
                 'PAMAP2_Dataset/Protocol/subject102.dat',
                 'PAMAP2_Dataset/Protocol/subject103.dat',
                 'PAMAP2_Dataset/Protocol/subject104.dat',
                 'PAMAP2_Dataset/Protocol/subject105.dat',
                 'PAMAP2_Dataset/Protocol/subject106.dat',
                 'PAMAP2_Dataset/Protocol/subject107.dat',
                 'PAMAP2_Dataset/Protocol/subject108.dat',
                 'PAMAP2_Dataset/Protocol/subject109.dat' ]

subjectID = [1,2,3,4,5,6,7,8,9]

activityIDdict = {0: 'transient',
              1: 'lying',
              2: 'sitting',
              3: 'standing',
              4: 'walking',
              5: 'running',
              6: 'cycling',
              7: 'Nordic_walking',
              9: 'watching_TV',
              10: 'computer_work',
              11: 'car driving',
              12: 'ascending_stairs',
              13: 'descending_stairs',
              16: 'vacuum_cleaning',
              17: 'ironing',
              18: 'folding_laundry',
              19: 'house_cleaning',
              20: 'playing_soccer',
              24: 'rope_jumping' }

colNames = ["timestamp", "activityID","heartrate"]

IMUhand = ['handTemperature', 
           'handAcc16_1', 'handAcc16_2', 'handAcc16_3', 
           'handAcc6_1', 'handAcc6_2', 'handAcc6_3', 
           'handGyro1', 'handGyro2', 'handGyro3', 
           'handMagne1', 'handMagne2', 'handMagne3',
           'handOrientation1', 'handOrientation2', 'handOrientation3', 'handOrientation4']

IMUchest = ['chestTemperature', 
           'chestAcc16_1', 'chestAcc16_2', 'chestAcc16_3', 
           'chestAcc6_1', 'chestAcc6_2', 'chestAcc6_3', 
           'chestGyro1', 'chestGyro2', 'chestGyro3', 
           'chestMagne1', 'chestMagne2', 'chestMagne3',
           'chestOrientation1', 'chestOrientation2', 'chestOrientation3', 'chestOrientation4']

IMUankle = ['ankleTemperature', 
           'ankleAcc16_1', 'ankleAcc16_2', 'ankleAcc16_3', 
           'ankleAcc6_1', 'ankleAcc6_2', 'ankleAcc6_3', 
           'ankleGyro1', 'ankleGyro2', 'ankleGyro3', 
           'ankleMagne1', 'ankleMagne2', 'ankleMagne3',
           'ankleOrientation1', 'ankleOrientation2', 'ankleOrientation3', 'ankleOrientation4']

columns = colNames + IMUhand + IMUchest + IMUankle  #all columns in one list

len(columns)


データファイルを読み込む．データファイルは，スペースで区切られているのでスペースごとに1行を読み込み，レコードを生成してデータフレームに保存している（保存の際に，ファイル名の数字部分から被験者ID：subject_id を生成して，データに付加している）．

In [ ]:
dataCollection = pd.DataFrame()
for file in list_of_files:
    procData = pd.read_table(file, header=None, sep=r'\s+')
    procData.columns = columns
    procData['subject_id'] = int(file[-5])
    dataCollection = pd.concat([dataCollection, procData], ignore_index=True)

dataCollection.reset_index(drop=True, inplace=True)
dataCollection.head(10)

上記のデータには，クリーニングしなければならないデータがある．たとえば，activityID が 0 のデータは，被験者が特定の活動をしていないことを示している．またいくつかの欠損値がある．

### データクレンジング

**PerformedActivitiesSummary.pdf** ファイルを参照すると，すべての被験者がすべての活動を行ったわけではない．したがって，欠損値が存在する．また，activityID が0 のデータは特定の活動をしていない．この値が変化していても，分析対象からは外す．次の順で実施するクレンジング用の関数を作成．
##### ・未使用のオリエンテーションデータレコード（3か所につき4種類ずつ）を削除
##### ・activityID が 0 のデータを削除
##### ・データ全体を数値化．数値化できないものは NaN に変換
##### ・NaN データ（欠損データ）を，前後のデータから補間（上端の欠損値はそのままになる）

In [ ]:
def dataCleaning(dataCollection):
        dataCollection = dataCollection.drop(['handOrientation1', 'handOrientation2', 'handOrientation3', 'handOrientation4',
                                             'chestOrientation1', 'chestOrientation2', 'chestOrientation3', 'chestOrientation4',
                                             'ankleOrientation1', 'ankleOrientation2', 'ankleOrientation3', 'ankleOrientation4'],
                                             axis = 1)  # removal of orientation columns as they are not needed
        dataCollection = dataCollection.drop(dataCollection[dataCollection.activityID == 0].index) #removal of any row of activity 0 as it is transient activity which it is not used
        dataCollection = dataCollection.apply(pd.to_numeric, errors = 'coerce') #removal of non numeric data in cells
        dataCollection = dataCollection.interpolate() #removal of any remaining NaN value cells by constructing new data points in known set of data points
        
        return dataCollection

In [ ]:
dataCol = dataCleaning(dataCollection)

In [ ]:
dataCol.reset_index(drop = True, inplace = True)
dataCol.head(10)

In [ ]:
dataCol.isnull().sum()

NaN のまま残っているのは，heartrate の 4 データであることが分かる（1 人目のデータの最初の4データ）．ここでは，5 行目以降のデータから同様に 100 を設定することが妥当そうなので，単純に 100 を代入する．

In [ ]:
for i in range(0,4):
    dataCol.loc[i, "heartrate"]=100

これにより，欠損値はなくなる．

In [ ]:
dataCol.isnull().sum()

## 探索的データ分析

#### 教師データとテストデータに分割

データ分割の前に，データ数の偏りをなるべくなくす必要がある．各クラスの重みがアンバランスであれば，サンプリングを行う必要がある．クラスの割合が，80% 対 20% を超える場合は，明らかに不均衡．ここではそのような状態でないかをまず確認する． 

activityID ごとのデータ件数を可視化すると，次のようになる．

In [ ]:
dataCol['activityID'].value_counts().plot(kind = "bar",figsize = (12,6))
plt.show()

これより，全体の80% を超えるクラスはなさそうなので，そのまま利用する．教師データとテストデータに分割．

In [ ]:
train_df = dataCol.sample(frac=0.8, random_state=1)
test_df = dataCol.drop(train_df.index)

データが用意できていそうなことを，確認する．教師データ，テストデータそれぞれを表示．分布が大きく変わっていなければよい．

In [ ]:
pd.options.display.float_format = '{:.2f}'.format
train_df.describe()

In [ ]:
test_df.describe()

心拍数は，被験者の状態を追跡するための重要な要素なので，どれくらいの分布をしているかを箱ひげ図で可視化．そのまえに，箱ひげ図や後で出てくるヒートマップを描画するライブラリをインストール．

In [ ]:
import seaborn as sns

fig, ax = plt.subplots(figsize=(4,4))
plt.title("Heart Rate ")
ax = sns.boxplot(y=train_df["heartrate"])

心拍数の箱ひげ図より，心拍数が180 を超える外れ値が存在していることがわかる．これらは非常に高い心拍数で，非常に激しいアクティビティや異常値である可能性がある．

最も負担の大きいアクティビティを見つけるために，各アクティビティごとの平均心拍数を示す棒グラフを描画．どのアクティビティが最も心拍数が高いか，つまり最も負担が大きいアクティビティを見つけることに利用できる．

In [ ]:
df_hr_act = train_df['heartrate'].groupby(train_df['activityID']).mean()
df_hr_act.index = df_hr_act.index.map(activityIDdict)
df_hr_act.plot(kind='bar')

この棒グラフからは，ランニングと縄跳びが心拍数への負荷が高そうなことが分かる．それらが他の活動に対して有意差があるかどうかは，後で確認する．

次にデータの異常や相関関係を調べるために，相関係数を求めてからヒートマップを作成する．

In [ ]:
from pandas.plotting import scatter_matrix
df_corr = train_df.corr()
df_corr = df_corr.drop(['activityID'], axis = 1)

f, ax = plt.subplots(figsize=(15, 10))
sns.heatmap(df_corr, mask=np.zeros_like(df_corr, dtype=np.bool), cmap = "BrBG",ax=ax)
plt.show()

このヒートマップからは次のことが分かる（次は一例）：
##### ・ジャイロスコープのデータはほかのどのデータともあまり関係していないように見え，この分析では不要．
##### ・心拍数と各部位の加速度計には相関がある．足首の加速度計とは正の相関，それ以外の部分とは負の相関．
##### ・心拍数と各部位の磁力計には相関がある．足首の磁力計とは負の相関，それ以外の部分とは正の相関．
##### → 上半身の身体活動は心拍数を上げて，下半身は下げる？

## 仮説の検定

最も負荷の高い活動は，ランニングと縄跳び．それらが他の活動と差があるかどうかを検定する．

**帰無仮説: **
- h0 : 負荷の高い活動の心拍数の平均値は，すべての活動に対して有意な差がない

**対立仮説: **

- h1 : 負荷の高い活動の心拍数の平均値は，すべての活動に対して大きく異なる

まず，ランニング(5) と縄跳び(24) のデータを取り出す．

In [ ]:
running_data = train_df.loc[(train_df["activityID"] == 5)]
ropejumping_data = train_df.loc[(train_df["activityID"] == 24)]
cumbersome_data = pd.concat([running_data,ropejumping_data])

それ以外のデータを取り出す．

In [ ]:
other_data = train_df.loc[(train_df["activityID"] != 5)]
other_data = other_data.loc[(other_data["activityID"] != 24)]

t検定を行い，p値を算出する．

In [ ]:
import scipy.stats
stat, pValue = scipy.stats.ttest_ind(cumbersome_data['heartrate'], other_data['heartrate'], equal_var=False)

In [ ]:
if pValue > 0.1:
    print("p値は", pValue, "で，対立仮説 h1 が棄却されました．負荷の高い活動の心拍数の平均値は，すべての活動に対して有意な差がありません．")
else:
    print("p値は", pValue, "で，帰無仮説 h0 が棄却されました．負荷の高い活動の心拍数の平均値は，すべての活動に対して大きく異なります．")